<a href="https://colab.research.google.com/github/reginasar/ULL_2023-main/hands-on/CAE_Star_Galaxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Unsupervised Star-Galaxy Separation with CAEs

The goal of this tutorial is to build an unsupervised star-galaxy separation using Convolutional Variational Auto-Encoders. 

We train a convolutional autoencoder (CAE) with stamps of images of stars and galaxies (without any labels). We then explore the latent space learned by the CAE and check tha the CAE has learned that there were 2 different populations in the dataset.

The dataset used for training is taken from COSMOS.

---



#### Before we start, make sure to open this Colab notebook in "PlayGround Mode" (top left) and to change the Runtime type to GPU by navigating to the toolbar and clicking Runtime -> Change runtime type and then changing Hardware accelerator to GPU

---

In [1]:
import os
import sys
import tarfile
from urllib.request import urlretrieve
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_datasets as tfds


import numpy as np
import matplotlib.pyplot as plt
import pdb
!pip install umap-learn[plot]
!pip install holoviews
!pip install -U ipykernel
import umap

import tensorflow_probability as tfp


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions

%pylab inline

ModuleNotFoundError: No module named 'tensorflow'

## Data download and preparation

Before mounting the drive click on [this folder](https://drive.google.com/drive/folders/1PcftgBzBySo1Ync-Wdsp9arTCJ_MfEPE?usp=sharing) and add it to your google drive by following these steps:

*   Go to your drive 
*   Find shared folder ("Shared with me" link)
*   Right click it
*   Click Add to My Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import glob
#glob.glob('/content/drive/MyDrive/*')

### Load data

In [ ]:
pathinData="/content/drive/MyDrive/star-galaxy"


# donwload feature vector and labels
X_ML = np.load(pathinData+'/x_train.npy')
#morphological class
Y_ML = np.load(pathinData+'/y_train.npy') 
#X_ML=X_ML-np.min(X_ML)
#print(X_ML.min())
#X_ML=X_ML/np.max(X_ML)
X_ML=(X_ML[:,30-14:30+14,30-14:30+14,:])

for i in range(len(X_ML)):
  X_ML[i,:,:]=(X_ML[i,:,:]-np.mean(X_ML[i,:,:]))/np.std(X_ML[i,:,:])

x_train=X_ML[0:int(len(X_ML)*3/5),:,:]
x_val=X_ML[int(len(X_ML)*3/5):int(len(X_ML)*4/5),:,:]
x_test=X_ML[int(len(X_ML)*4/5)::,:,:]

y_train=Y_ML[0:int(len(X_ML)*3/5)]
y_val=Y_ML[int(len(X_ML)*3/5):int(len(X_ML)*4/5)]
y_test=Y_ML[int(len(X_ML)*4/5)::]

#print((Y_ML==0).sum(), (Y_ML==1).sum())

### Plot some random example
Run multiple times to see more than one example. Class0 are stars, Class1 are galaxies.

In [ ]:
randomized_inds_train = np.random.permutation(len(x_train))

fig = plt.figure()
for i,j in zip(randomized_inds_train[0:4],range(4)):
  ax = fig.add_subplot(2, 2, j+1)
  im = ax.imshow(X_ML[i,:,:,0])
  plt.title('$Class$='+str(y_train[i]))
  fig.tight_layout() 
  fig.colorbar(im)

## Model Setup

The following cells set up the model in TensorFlow. We have set up a very basic Convolutional Encoder-Decoder. It does not perform super well for this problem. I encourage you to try to modify this network to achieve better results. You should in particular try to reduce more the dimension of the bottleneck (end of encoder).

In [ ]:
def pack_images(images, rows, cols):
    """Helper utility to make a field of images."""
    shape = tf.shape(images)
    width = shape[-3]
    height = shape[-2]
    depth = shape[-1]
    images = tf.reshape(images, (-1, width, height, depth))
    batch = tf.shape(images)[0]
    rows = tf.minimum(rows, batch)
    cols = tf.minimum(batch // rows, cols)
    images = images[:rows * cols]
    images = tf.reshape(images, (rows, cols, width, height, depth))
    images = tf.transpose(images, [0, 2, 1, 3, 4])
    images = tf.reshape(images, [1, rows * width, cols * height, depth])
    return images


def image_tile_summary(name, tensor, rows=8, cols=8):
    tf.summary.image(name, pack_images(tensor, rows, cols), max_outputs=1)


input_shape = (28,28,1)
encoded_size = 16  # THIS IS THE SIZE OF THE BOTTLENECK --> FEEL FREE TO CHANGE
# AND EXPLORE
base_depth = 32


##################### CAE #####################

encoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=input_shape),
    #tfkl.Lambda(lambda x: tf.cast(x, tf.float32) - 0.5),
    tfkl.Conv2D(base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(4 * encoded_size, 7, strides=1,
                padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Flatten(),
    tfkl.Dense(encoded_size),
])



decoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=[encoded_size]),
    tfkl.Reshape([1, 1, encoded_size]),
    tfkl.Conv2DTranspose(2 * base_depth, 7, strides=1,
                         padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(filters=1, kernel_size=5, strides=1,
                padding='same', activation=None),
])


cae = tfk.Model(inputs=encoder.inputs,outputs=decoder(encoder.outputs[0]))


################## VAE #################

prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                        reinterpreted_batch_ndims=1)

vencoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=input_shape),
    #tfkl.Lambda(lambda x: tf.cast(x, tf.float32) - 0.5),
    tfkl.Conv2D(base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(4 * encoded_size, 7, strides=1,
                padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Flatten(),
    tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
               activation=None),
    tfpl.MultivariateNormalTriL(
        encoded_size,
        activity_regularizer=tfpl.KLDivergenceRegularizer(prior,weight=0.6)),
])

vdecoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=[encoded_size]),
    tfkl.Reshape([1, 1, encoded_size]),
    tfkl.Conv2DTranspose(2 * base_depth, 7, strides=1,
                         padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(filters=1, kernel_size=5, strides=1,
                padding='same', activation=None),
    tfkl.Flatten(),
    
    tfkl.Dense(tfpl.IndependentNormal.params_size(input_shape),activation=None),
    #tfpl.IndependentNormal(input_shape, tfd.Normal.mean),
    tfpl.IndependentNormal(input_shape, tfd.Normal.sample),
    #tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits),
])

vae = tfk.Model(inputs=vencoder.inputs, outputs=vdecoder(vencoder.outputs[0]))
       

       




In [ ]:
encoder.summary()
decoder.summary()

## Training the model

In [ ]:
#Define output path - This folder will contain the trained model
pathout='star-galaxy/models/cae1'
#pathout='star-galaxy/models/vae1'

#Set RESET=True to delete all previous runs of the same model
RESET=False
if RESET:
  tf.summary.FileWriterCache.clear()
  os.system("rm -rf "+ pathout)

Train first for 1000 steps. This is just a trick to run TensorBoard on the notebook and follow the training.

In [ ]:
cae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss='mse')

hist = cae.fit(x_train,x_train,epochs=20,validation_data=(x_val, x_val))


#negloglik = lambda x, rv_x: -rv_x.log_prob(x)

#vae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
#            loss=negloglik)

#hist = vae.fit(x_train,x_train,epochs=20,validation_data=(x_val, y_val))

In [ ]:
#plt.plot(hist.history['loss'])
#plt.plot(hist.history['val_loss'],color='red')
#plt.xlabel('Epochs')
#plt.ylabel('Loss')

## Predictions and evaluation

In [ ]:
#predict of the first 10 examples
pred = cae(x_test[0:10])

fig = plt.figure()
for i,j in zip(range(10),range(4)):
  ax = fig.add_subplot(2, 2, j+1)
  im = ax.imshow(x_val[i,:,:,0])
  plt.title('$Class$='+str(y_val[i]))
  fig.tight_layout() 
  fig.colorbar(im)

fig = plt.figure()
for i,j in zip(range(10),range(4)):
  ax = fig.add_subplot(2, 2, j+1)
  im = ax.imshow(pred[i,:,:,0])
  plt.title('$Class$='+str(y_val[i]))
  fig.tight_layout() 
  fig.colorbar(im)

In [ ]:
#xhat = vae(x_val[0:10])
#assert isinstance(xhat, tfd.Distribution)

#print(x_test.shape)
#pdb.set_trace()

In [ ]:
# Now, let's generate ten never-before-seen galaxies and stars.
#z = prior.sample(10)
#xtilde = decoder(z)
#assert isinstance(xtilde, tfd.Distribution)

In [ ]:
#fig = plt.figure()
#for i,j in zip(range(10),range(4)):
#  ax = fig.add_subplot(2, 2, j+1)
#  im = ax.imshow(xtilde.sample()[i,:,:,0])
#  fig.tight_layout() 
#  fig.colorbar(im)

In [ ]:
# Now, let's explore the latent space
z = encoder(x_test)
print("shape of encoded space:", z.shape)

# since it is 16 dimensions, we are trying to visualize it with UMAP
reducer = umap.UMAP()
embedding = reducer.fit_transform(z)
embedding.shape

plt.xlabel("UMAP1", fontsize=20)
plt.ylabel("UMAP2", fontsize=20)

plt.scatter(embedding[y_test==1,1],embedding[y_test==1,0],color='blue',s=1,label='Galaxies')
plt.scatter(embedding[y_test==0,1],embedding[y_test==0,0],color='red',s=1,label='Stars')
plt.legend(fontsize=14)



## Clustering with K-Means / GMMs





Can you run a GMM in the encoded space? Look at the scikit learn documentation for GMMs. Use it to compute a classification accuracy. What is the fraction of stars classified as galaxies with this approach. The requirement for Euclid is less than 1%... 

